In [1]:
from utils import process_tweets,lookup,build_freqs
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [2]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [3]:
freqs=build_freqs(train_x,train_y)

In [4]:
def NavieBayes(freqs,train_x,train_y):
    loglikelihood = {}
    logprior = 0
    vocab = set([i[0] for i in freqs.keys()])
    V = len(vocab)
    N_pos,N_neg = 0,0
    for pair in freqs.keys():
        if pair[1] == 1: N_pos += freqs[pair]
        else: N_neg += freqs[pair]
    D = len(train_y)
    Dpos = int(sum(train_y))
    Dneg = D-Dpos
    logprior = np.log(Dpos) - np.log(Dneg)
    for word in vocab:
        freqpos = lookup(freqs,word,1)
        freqneg = lookup(freqs,word,0)
        pwpos = (freqpos+1)/(V+N_pos)
        pwneg = (freqneg+1)/(V+N_neg)
        loglikelihood[word] = np.log(pwpos) - np.log(pwneg)
    return logprior,loglikelihood

In [5]:
logprior, loglikelihood = NavieBayes(freqs,train_x,train_y)
print(logprior)
print(len(loglikelihood))

0.0
9084


In [6]:
def predict(tweet,loglikelihood,logprior):
    words = process_tweets(tweet)
    p = 0
    p = p + logprior
    for word in words:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p

In [7]:
my_tweet = 'great great great.'
p = predict(my_tweet,loglikelihood,logprior)
print('The expected output is', p)

The expected output is 6.413549166620699


In [8]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0 
    y_hats = []
    for tweet in test_x:
        if predict(tweet,loglikelihood,logprior) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
    error = np.mean(np.absolute(y_hats-test_y))
    accuracy = 1-error
    return accuracy

In [9]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9940


In [10]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    p = predict(tweet, loglikelihood, logprior)
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.15
I am bad -> -1.29
this movie should have been great. -> 2.14
great -> 2.14
great great -> 4.28
great great great -> 6.41
great great great great -> 8.55
